In [627]:
from selenium import webdriver
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
import math

# 1. Get data

In [2]:
total_list = []

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.amazon.com/s?i=toys-and-games&bbn=165793011&rh=n%3A165793011%2Cp_n_age_range%3A165890011&dc&page=42&qid=1628625940&rnid=165794011&ref=sr_pg_42')
url_list = []
next_page = [-1]
page = 42
times = 0
while times<=400:
    time.sleep(2)
    item = driver.find_elements_by_class_name('s-image')
    j = 0
    item_n = len(item)
    print(item_n)
    while j<item_n:
        time.sleep(1)
        item[j].click()
        #try:
        #    
        #except:
        #    continue
        curr_url = driver.current_url
        if curr_url not in url_list:
            url_list.append(str(curr_url))
        driver.back()
        j+=1
        item = driver.find_elements_by_class_name('s-image')
        item_n = len(item)
    try:
        next_page = driver.find_elements_by_link_text("Next")
        next_page[0].click()
        page+=1
    except Exception as e:
        if len(next_page)==0:
            while len(next_page)==0:
                try:
                    driver.refresh()
                    next_page = driver.find_elements_by_link_text("Next")
                    next_page1 = driver.find_elements_by_link_text(str(page))
                    if len(next_page)!=0:
                        next_page[0].click()
                        page+=1
                    elif len(next_page1)!=0:
                        next_page1[0].click()
                        next_page = next_page1
                        page+=1
                    print(len(next_page))
                except Exception as e:
                    print(next_page)
                    print(next_page1)
                    print('There is not next page')
                next_page = [1]
        else:
            for next_p in next_page:
                print('111111')
                n_p = next_p.get_attribute('class')
                if 's-pagination-item' in n_p.split(' ') or 's-pagination-next' in n_p.split(' ') or 's-pagination-button' in n_p.split(' ') or 's-pagination-separator' in n_p.split(' '):
                    next_page[0].click()
                    page+=1
                next_page = [1]
        
    times+=1
driver.quit()

In [14]:
page

74

In [15]:
len(url_list)

773

In [16]:
url_list = list(set(url_list))

In [17]:
len(url_list)

773

In [18]:
ur = pd.DataFrame({'url':url_list})

In [19]:
ur.to_csv('url5.csv')

# 2. merge data from csv

In [33]:
data_url = []
for i in range(1, 6):
    temp = pd.read_csv(f'url{i}.csv')
    data_url.append(temp)
url_list = pd.concat(data_url)

In [34]:
url_list = list(url_list['url'])

In [ ]:
title_list = []
number_rating_list = []
price_list = []
describe1_list = []
describe2_list = []
describe3_list = []
describe4_list = []
describe5_list = []
Product_Dimensions_list = []
Item_Weight_list = []
ASIN_list = []
Manufacturer_recommended_age_list = []
Best_Sellers_Rank_list = []
Release_date_list = []

for i in range(len(url_list)):
#for i in range(3):
    driver = webdriver.Chrome()
    driver.get(url_list[i])
    time.sleep(1)
    try:
        title = driver.find_element_by_xpath('//*[@id="productTitle"]').text
    except Exception:
        title = np.nan
    try:
        number_rating = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
    except Exception:
        number_rating = np.nan
    try:
        price = driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text
    except Exception:
        price = np.nan
    try :
        describe1 = driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul/li[2]/span').text
    except Exception:
        describe1 = np.nan
    try:
        describe2 = driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul/li[3]/span').text
    except Exception:
        describe2 = np.nan
    try:
        describe3 = driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul/li[4]/span').text
    except Exception:
        describe3 = np.nan
    try:
        describe4 = driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul/li[5]/span').text
    except Exception:
        describe4 = np.nan
    try:
        describe5 = driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul/li[6]/span').text
    except Exception:
        describe5 = np.nan
    try:
        rows=1+len(driver.find_elements_by_xpath('//*[@id="productDetails_detailBullets_sections1"]/tbody/tr'))
    except Exception:
        rows = 1
    th_list = []
    td_list = []
    try:
        for r in range(1, rows):
            th = driver.find_element_by_xpath(f'//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[{r}]/th').text
            td = driver.find_element_by_xpath(f'//*[@id="productDetails_detailBullets_sections1"]/tbody/tr[{r}]/td').text
            th_list.append(th)
            td_list.append(td)
        table = pd.DataFrame({'th':th_list, 'td':td_list})
        table = table.set_index('th')
    except Exception:
        table = pd.DataFrame()
        
    try:
        Product_Dimensions = table.loc['Product Dimensions'][0]
    except Exception:
        Product_Dimensions = np.nan
    try:
        Item_Weight = table.loc['Item Weight'][0]
    except Exception:
        Item_Weight = np.nan
    try:
        ASIN = table.loc['ASIN'][0]
    except Exception:
        ASIN = np.nan   
    try:
        Manufacturer_recommended_age = table.loc['Manufacturer recommended age'][0]
    except Exception:
        Manufacturer_recommended_age = np.nan  
    try:
        Best_Sellers_Rank = table.loc['Best Sellers Rank'][0]
    except Exception:
        Best_Sellers_Rank = np.nan 
    try:
        Release_date = table.loc['Release date'][0]
    except Exception:
        Release_date = np.nan     

    title_list.append(title)
    number_rating_list.append(number_rating) 
    price_list.append(price) 
    describe1_list.append(describe1) 
    describe2_list.append(describe2) 
    describe3_list.append(describe3) 
    describe4_list.append(describe4) 
    describe5_list.append(describe5) 
    Product_Dimensions_list.append(Product_Dimensions) 
    Item_Weight_list.append(Item_Weight) 
    ASIN_list.append(ASIN) 
    Manufacturer_recommended_age_list.append(Manufacturer_recommended_age) 
    Best_Sellers_Rank_list.append(Best_Sellers_Rank) 
    Release_date_list.append(Release_date) 
    driver.quit()

In [36]:
amazon_data = pd.DataFrame({'ASIN_list':ASIN_list, 'title_list':title_list,  'price_list':price_list, 'describe1_list':describe1_list, 'describe2_list':describe2_list, 'describe3_list':describe3_list, 'describe4_list':describe4_list, 'describe5_list':describe5_list, 'Product_Dimensions_list':Product_Dimensions_list, 'Item_Weight_list':Item_Weight_list, 'Manufacturer_recommended_age_list':Manufacturer_recommended_age_list,  'Release_date_list':Release_date_list, 'number_rating_list':number_rating_list,'Best_Sellers_Rank_list':Best_Sellers_Rank_list})

In [45]:
#amazon_data.to_csv('amazon_data.csv', index=False)

In [326]:
amazon_data = pd.read_csv('amazon_data.csv')

## 3.clean data

In [327]:
n_item = len(amazon_data)

In [328]:
url_list_1 = url_list[0:n_item]

In [329]:
amazon_data['url'] = url_list_1

In [330]:
amazon_data

,ASIN_list,title_list,price_list,describe1_list,describe2_list,describe3_list,describe4_list,describe5_list,Product_Dimensions_list,Item_Weight_list,Manufacturer_recommended_age_list,Release_date_list,number_rating_list,Best_Sellers_Rank_list,url
0,B07N72DVHW,"Crayola Paw Patrol Color Wonder, Mess Free Col...",$6.99,COLOR WONDER SET: Features 18 Paw Patrol Color...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,MESS FREE COLORING: Colors appear only on spec...,PAW PATROL CHARACTERS: Pairs well with other P...,"SAFE & NONTOXIC: Gift for girls and boys, ages...",NaN,NaN,NaN,NaN,"7,662 ratings",#29 in Toys & Games (See Top 100 in Toys & Gam...,https://www.amazon.com/Crayola-Color-Wonder-Co...
1,B07B6ZN7P8,LeapFrog Learning Friends 100 Words Book,$15.99,"Meet learning friends Turtle, Tiger and Monkey...","Word categories include: pets, animals, food, ...",Touching the words on the pages plays the word...,"Words, songs and instructions can be heard in ...",Intended for ages 18+ months; requires 2 AA ba...,9.4 x 9.2 x 1.9 inches,1.52 pounds,18 months - 4 years,"April 1, 2018","72,915 ratings",#9 in Toys & Games (See Top 100 in Toys & Game...,https://www.amazon.com/LeapFrog-Learning-Frien...
2,B00004UBH2,"Crayola Washable Kids Paint, 6 Count, Kids At ...",$7.29,CLASSIC COLORS: Washable Kids’ Paint in six di...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,WASHABLE COLORS: From washable markers & washa...,CRAYOLA PAINTS are perfect for use at-home or ...,WORRY-FREE PAINTING: Crayola Washable Kids Pai...,3.56 x 8.88 x 2.56 inches,1 pounds,4 years and up,"January 1, 2011","26,670 ratings",#93 in Toys & Games (See Top 100 in Toys & Gam...,https://www.amazon.com/Crayola-Washable-Classi...
3,B083XNCRS3,"Bluey and Friends 4 Pack of 2.5-3"" Poseable Fi...",$14.99,Collect Bluey's whole family with the Bluey & ...,"In this pack you will find Bluey, her little s...",All 4 figures in the Family 4-Pack are fully p...,The perfect size for pre-schooler hands to pla...,There are so many of Bluey's family and friend...,3.24 x 2.1 x 2.05 inches,2.82 ounces,36 months - 15 years,"March 30, 2020","9,240 ratings",#43 in Toys & Games (See Top 100 in Toys & Gam...,https://www.amazon.com/Bluey-Family-Bingo-Figu...
4,B07TT6664Z,"Scientoy Fidget Toy Set, 35 Pcs Sensory Toy fo...",$17.99,◆【Super-Value 35 Packed Fidget Toys】 Scientoy ...,"◆【Respectable quality,Safe】Made of high-qualit...","◆【Gift Packed,Fidget Toys for Sensory kids】Sci...","◆【Easy to Carry, Suitable for All Occasions】Wi...",◆ 【Satisfaction Service】 When you purchase our...,NaN,1.59 pounds,3 years and up,NaN,"4,379 ratings",#101 in Toys & Games (See Top 100 in Toys & Ga...,https://www.amazon.com/Scientoy-Blocks/dp/B07T...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,B077V2VLZM,"Crayola My First, Tripod Washable Markers for ...",$8.30,This set features 8 Ultra-Clean Washable Marke...,The triangular marker shape is easy to grasp w...,These toddler markers won’t roll off the table...,Includes 8 different washable Crayola marker c...,"Safe for toddlers, 24 months & up.",0.94 x 7.25 x 7.5 inches,5.6 ounces,24 months - 10 years,"December 15, 2017","2,017 ratings","#5,606 in Toys & Games (See Top 100 in Toys & ...",https://www.amazon.com/Crayola-Tripod-Washable...
873,B085T72CM3,UMIKU 100 Pack Pencil Erasers Animal Erasers f...,$32.99,🍟{100 HUGE PACK ERASER COLLECTION}🍩: 3D Mini e...,🍒{FUN & EDUCATIONAL}🍑: Best eraser toys for li...,🍕{FIT FOR VARIOUS OCCASIONS}🍆: Perfect pencil ...,🐳{BEST KIDS GIFTS}🍔: Ideal gifts for 3 4 5+ ye...,🍌{PUZZLE PENCIL ERASERS}🍓: Super kawaii animal...,NaN,NaN,NaN,NaN,456 ratings,"#3,063 in Toys & Games (See Top 100 in Toys & ...",https://www.amazon.com/UMIKU-Erasers-Vegetable...
874,B00C2P6VWW,Little Tikes Activity Garden Playhouse for Bab...,$70.92,EVERYTHING IS BLOOMING. Multi-functional activ...,EVERYTHING IS BLOOMING. Multi-functional activ...,ON-THE-GO PLAY. Removable take-along play pane...,INTERACTIVE FEATURES. Window shutters open and...

In [331]:
amazon_data.price_list = amazon_data.price_list.apply(lambda x:x[1:] if x!=np.nan and type(x)==str else x)

In [718]:
amazon_data1 = amazon_data.copy()

In [719]:
def extract_rank_number(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        pattern = re.compile(r'#[0-9]+')
        result = re.findall(pattern, x)
        return int(result[0][1:])
     else:
        return np.nan

In [720]:
def extract_rating_number(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        pattern = re.compile(r'[0-9,]+')
        result = re.findall(pattern, x)
        result1 = result[0].replace(',','')
        return int(result1)
     else:
        return np.nan

In [721]:
def transferTime(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        return(pd.to_datetime(x))
     else:
        return np.nan

In [722]:
def extract_recomend_year(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        pattern = re.compile(r'[0-9]+\syears')
        year = re.findall(pattern, x)
        pattern1 = re.compile(r'[0-9]+\smonths')
        months = re.findall(pattern1, x)
        if len(months)!=0:
            return float(months[0][0:-7])/12
        elif len(year)!=0:
            return year[0][0:-6]
        else:
            return np.nan
     else:
        return np.nan

In [723]:
def extract_Item_Weight(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        return(float(x[0:-7]))
     else:
        return np.nan

In [724]:
def extract_Product_Dimensions(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        x = x[0:-7]
        x = x.split(' x ')
        return(x)
     else:
        return np.nan

In [725]:
def remove_stop_word(x):
     if x == np.nan:
        return np.nan
     elif type(x)==str:
        tokenizer = nltk.data.load('/Users/zeyu/nltk_data/tokenizers/punkt/english.pickle')
        x = x.lower()
        x = tokenizer.tokenize(x)
        sw = stopwords.words('english')
        x = {w for w in x if not w in sw}
        return x
     else:
        return np.nan

In [726]:
a = '9.4 x 9.2 x 1.9 inches'
a[0:-7]

'9.4 x 9.2 x 1.9'

In [727]:
amazon_data1.Best_Sellers_Rank_list = amazon_data1.Best_Sellers_Rank_list.apply(lambda x: extract_rank_number(x))

In [728]:
amazon_data1.number_rating_list = amazon_data1.number_rating_list.apply(lambda x: extract_rating_number(x))

In [729]:
amazon_data1.Release_date_list = amazon_data1.Release_date_list.apply(lambda x: transferTime(x))

In [730]:
amazon_data1.Manufacturer_recommended_age_list = amazon_data1.Manufacturer_recommended_age_list.apply(lambda x: extract_recomend_year(x))

In [731]:
amazon_data1.Item_Weight_list = amazon_data1.Item_Weight_list.apply(lambda x: extract_Item_Weight(x))

In [732]:
amazon_data1.Product_Dimensions_list = amazon_data1.Product_Dimensions_list.apply(lambda x: extract_Product_Dimensions(x))

In [733]:
#amazon_data1.title_list = amazon_data1.title_list.apply(lambda x: remove_stop_word(x))

In [734]:
#amazon_data1.describe1_list = amazon_data1.describe1_list.apply(lambda x: remove_stop_word(x))

In [735]:
#amazon_data1.describe2_list = amazon_data1.describe2_list.apply(lambda x: remove_stop_word(x))

In [736]:
#amazon_data1.describe3_list = amazon_data1.describe3_list.apply(lambda x: remove_stop_word(x))

In [737]:
#amazon_data1.describe4_list = amazon_data1.describe4_list.apply(lambda x: remove_stop_word(x))

In [738]:
#amazon_data1.describe5_list = amazon_data1.describe5_list.apply(lambda x: remove_stop_word(x))

In [739]:
amazon_data1

,ASIN_list,title_list,price_list,describe1_list,describe2_list,describe3_list,describe4_list,describe5_list,Product_Dimensions_list,Item_Weight_list,Manufacturer_recommended_age_list,Release_date_list,number_rating_list,Best_Sellers_Rank_list,url
0,B07N72DVHW,"Crayola Paw Patrol Color Wonder, Mess Free Col...",6.99,COLOR WONDER SET: Features 18 Paw Patrol Color...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,MESS FREE COLORING: Colors appear only on spec...,PAW PATROL CHARACTERS: Pairs well with other P...,"SAFE & NONTOXIC: Gift for girls and boys, ages...",NaN,NaN,NaN,NaT,7662.0,29.0,https://www.amazon.com/Crayola-Color-Wonder-Co...
1,B07B6ZN7P8,LeapFrog Learning Friends 100 Words Book,15.99,"Meet learning friends Turtle, Tiger and Monkey...","Word categories include: pets, animals, food, ...",Touching the words on the pages plays the word...,"Words, songs and instructions can be heard in ...",Intended for ages 18+ months; requires 2 AA ba...,"[9.4, 9.2, 1.9]",1.52,1.5,2018-04-01,72915.0,9.0,https://www.amazon.com/LeapFrog-Learning-Frien...
2,B00004UBH2,"Crayola Washable Kids Paint, 6 Count, Kids At ...",7.29,CLASSIC COLORS: Washable Kids’ Paint in six di...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,WASHABLE COLORS: From washable markers & washa...,CRAYOLA PAINTS are perfect for use at-home or ...,WORRY-FREE PAINTING: Crayola Washable Kids Pai...,"[3.56, 8.88, 2.56]",1.00,4,2011-01-01,26670.0,93.0,https://www.amazon.com/Crayola-Washable-Classi...
3,B083XNCRS3,"Bluey and Friends 4 Pack of 2.5-3"" Poseable Fi...",14.99,Collect Bluey's whole family with the Bluey & ...,"In this pack you will find Bluey, her little s...",All 4 figures in the Family 4-Pack are fully p...,The perfect size for pre-schooler hands to pla...,There are so many of Bluey's family and friend...,"[3.24, 2.1, 2.05]",2.82,3.0,2020-03-30,9240.0,43.0,https://www.amazon.com/Bluey-Family-Bingo-Figu...
4,B07TT6664Z,"Scientoy Fidget Toy Set, 35 Pcs Sensory Toy fo...",17.99,◆【Super-Value 35 Packed Fidget Toys】 Scientoy ...,"◆【Respectable quality,Safe】Made of high-qualit...","◆【Gift Packed,Fidget Toys for Sensory kids】Sci...","◆【Easy to Carry, Suitable for All Occasions】Wi...",◆ 【Satisfaction Service】 When you purchase our...,NaN,1.59,3,NaT,4379.0,101.0,https://www.amazon.com/Scientoy-Blocks/dp/B07T...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,B077V2VLZM,"Crayola My First, Tripod Washable Markers for ...",8.30,This set features 8 Ultra-Clean Washable Marke...,The triangular marker shape is easy to grasp w...,These toddler markers won’t roll off the table...,Includes 8 different washable Crayola marker c...,"Safe for toddlers, 24 months & up.","[0.94, 7.25, 7.5]",5.60,2.0,2017-12-15,2017.0,5.0,https://www.amazon.com/Crayola-Tripod-Washable...
873,B085T72CM3,UMIKU 100 Pack Pencil Erasers Animal Erasers f...,32.99,🍟{100 HUGE PACK ERASER COLLECTION}🍩: 3D Mini e...,🍒{FUN & EDUCATIONAL}🍑: Best eraser toys for li...,🍕{FIT FOR VARIOUS OCCASIONS}🍆: Perfect pencil ...,🐳{BEST KIDS GIFTS}🍔: Ideal gifts for 3 4 5+ ye...,🍌{PUZZLE PENCIL ERASERS}🍓: Super kawaii animal...,NaN,NaN,NaN,NaT,456.0,3.0,https://www.amazon.com/UMIKU-Erasers-Vegetable...
874,B00C2P6VWW,Little Tikes Activity Garden Playhouse for Bab...,70.92,EVERYTHING IS BLOOMING. Multi-functional activ...,EVERYTHING IS BLOOMING. Multi-functional activ...,ON-THE-GO PLAY. Removable take-along play pane...,INTERACTIVE FEATURES. Window shutters open and...,TAP-A-TUNE PIANO. Colored chimes and buttons p...,"[25.51, 25, 33.24]",5.06,0.5,2013-06-28,6817.0,3.0,https://www.amazon.com/Little-Tikes-Activity-G...
875,NaN,Pack of 5 Kids Face Mask Set - Children’s 3 La...,NaN,Adjustable closure,Machine Wash,🐻 𝗧𝗥𝗜𝗣𝗟𝗘-𝗟𝗔𝗬𝗘𝗥 𝗣𝗥𝗢𝗧𝗘𝗖𝗧𝗜𝗢𝗡 - For efficient prot...,😸 𝗥𝗘𝗨𝗦𝗔𝗕𝗟𝗘 𝗔𝗡𝗗 𝗪𝗔𝗦𝗛𝗔𝗕𝗟𝗘 - To save you the extr...,🐸 𝗔𝗗𝗝𝗨𝗦𝗧𝗔𝗕𝗟𝗘 𝗙𝗜𝗧 𝗔𝗡𝗗 𝗖𝗢𝗠𝗙𝗢𝗥𝗧 - These kid’s fav...,NaN,NaN,NaN,NaT,241.0,NaN,https://www.amazon.com/Pack-Kids-Face-Mask-Set...


In [740]:
amazon_data1['last_time'] = (pd.to_datetime("now")-amazon_data1.Release_date_list)

In [741]:
amazon_data1

,ASIN_list,title_list,price_list,describe1_list,describe2_list,describe3_list,describe4_list,describe5_list,Product_Dimensions_list,Item_Weight_list,Manufacturer_recommended_age_list,Release_date_list,number_rating_list,Best_Sellers_Rank_list,url,last_time
0,B07N72DVHW,"Crayola Paw Patrol Color Wonder, Mess Free Col...",6.99,COLOR WONDER SET: Features 18 Paw Patrol Color...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,MESS FREE COLORING: Colors appear only on spec...,PAW PATROL CHARACTERS: Pairs well with other P...,"SAFE & NONTOXIC: Gift for girls and boys, ages...",NaN,NaN,NaN,NaT,7662.0,29.0,https://www.amazon.com/Crayola-Color-Wonder-Co...,NaT
1,B07B6ZN7P8,LeapFrog Learning Friends 100 Words Book,15.99,"Meet learning friends Turtle, Tiger and Monkey...","Word categories include: pets, animals, food, ...",Touching the words on the pages plays the word...,"Words, songs and instructions can be heard in ...",Intended for ages 18+ months; requires 2 AA ba...,"[9.4, 9.2, 1.9]",1.52,1.5,2018-04-01,72915.0,9.0,https://www.amazon.com/LeapFrog-Learning-Frien...,1228 days 03:45:44.695354
2,B00004UBH2,"Crayola Washable Kids Paint, 6 Count, Kids At ...",7.29,CLASSIC COLORS: Washable Kids’ Paint in six di...,AT HOME CRAFTS & INDOOR ACTIVITIES: Keep spiri...,WASHABLE COLORS: From washable markers & washa...,CRAYOLA PAINTS are perfect for use at-home or ...,WORRY-FREE PAINTING: Crayola Washable Kids Pai...,"[3.56, 8.88, 2.56]",1.00,4,2011-01-01,26670.0,93.0,https://www.amazon.com/Crayola-Washable-Classi...,3875 days 03:45:44.695354
3,B083XNCRS3,"Bluey and Friends 4 Pack of 2.5-3"" Poseable Fi...",14.99,Collect Bluey's whole family with the Bluey & ...,"In this pack you will find Bluey, her little s...",All 4 figures in the Family 4-Pack are fully p...,The perfect size for pre-schooler hands to pla...,There are so many of Bluey's family and friend...,"[3.24, 2.1, 2.05]",2.82,3.0,2020-03-30,9240.0,43.0,https://www.amazon.com/Bluey-Family-Bingo-Figu...,499 days 03:45:44.695354
4,B07TT6664Z,"Scientoy Fidget Toy Set, 35 Pcs Sensory Toy fo...",17.99,◆【Super-Value 35 Packed Fidget Toys】 Scientoy ...,"◆【Respectable quality,Safe】Made of high-qualit...","◆【Gift Packed,Fidget Toys for Sensory kids】Sci...","◆【Easy to Carry, Suitable for All Occasions】Wi...",◆ 【Satisfaction Service】 When you purchase our...,NaN,1.59,3,NaT,4379.0,101.0,https://www.amazon.com/Scientoy-Blocks/dp/B07T...,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,B077V2VLZM,"Crayola My First, Tripod Washable Markers for ...",8.30,This set features 8 Ultra-Clean Washable Marke...,The triangular marker shape is easy to grasp w...,These toddler markers won’t roll off the table...,Includes 8 different washable Crayola marker c...,"Safe for toddlers, 24 months & up.","[0.94, 7.25, 7.5]",5.60,2.0,2017-12-15,2017.0,5.0,https://www.amazon.com/Crayola-Tripod-Washable...,1335 days 03:45:44.695354
873,B085T72CM3,UMIKU 100 Pack Pencil Erasers Animal Erasers f...,32.99,🍟{100 HUGE PACK ERASER COLLECTION}🍩: 3D Mini e...,🍒{FUN & EDUCATIONAL}🍑: Best eraser toys for li...,🍕{FIT FOR VARIOUS OCCASIONS}🍆: Perfect pencil ...,🐳{BEST KIDS GIFTS}🍔: Ideal gifts for 3 4 5+ ye...,🍌{PUZZLE PENCIL ERASERS}🍓: Super kawaii animal...,NaN,NaN,NaN,NaT,456.0,3.0,https://www.amazon.com/UMIKU-Erasers-Vegetable...,NaT
874,B00C2P6VWW,Little Tikes Activity Garden Playhouse for Bab...,70.92,EVERYTHING IS BLOOMING. Multi-functional activ...,EVERYTHING IS BLOOMING. Multi-functional activ...,ON-THE-GO PLAY. Removable take-along play pane...,INTERACTIVE FEATURES. Window shutters open and...,TAP-A-TUNE PIANO. Colored chimes and buttons p...,"[25.51, 25, 33.24]",5.06,0.5,2013-06-28,6817.0,3.0,https://www.amazon.com/Little-Tikes-Activity-G...,2966 days 03:45:44.695354
875,NaN,Pack of 5 Kids Face Mask Set - Children’s 3 La...,NaN,Adjustable closure,Machine Wash,🐻 𝗧𝗥𝗜𝗣𝗟𝗘-𝗟𝗔𝗬𝗘𝗥 𝗣𝗥𝗢𝗧𝗘𝗖𝗧𝗜𝗢𝗡 - For efficient prot...,😸 𝗥𝗘𝗨𝗦𝗔𝗕𝗟𝗘 𝗔𝗡𝗗 𝗪𝗔𝗦𝗛𝗔𝗕𝗟𝗘 - To save you the extr...,🐸 𝗔𝗗𝗝𝗨𝗦𝗧𝗔𝗕𝗟𝗘 𝗙𝗜𝗧 𝗔𝗡𝗗 𝗖𝗢𝗠𝗙𝗢𝗥𝗧 - These kid’

## 4. K-nn

In [1323]:
x = amazon_data1[['price_list', 'Item_Weight_list', 'Product_Dimensions_list', 'Manufacturer_recommended_age_list', 'last_time', 'title_list','describe1_list','describe2_list','describe3_list','describe4_list','describe5_list']]
y = amazon_data1[['number_rating_list', 'Best_Sellers_Rank_list', 'url', 'ASIN_list']]

In [1324]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [1325]:
def knn(data, query, k, distance_fn):
    neighbor_distance = []
    for i in range(len(data)):
        example = data.iloc[i,:]
        distance = distance_fn(example, query)
        neighbor_distance.append(distance)
    return neighbor_distance
    #sorted_neighbor_distances_and_indics = sorted(neighbor_distance_and_indices)
    #k_nearest_distance_and_indices = sorted_neighbor_distances_and_indics
    #return k_nearest_distance_and_indices[0:k]

In [1326]:
def remove_stop_word(x):
     if x == np.nan:
        return np.nan
     elif type(x)==list:
        sw = stopwords.words('english')
        x = {w for w in x if not w in sw}
        return x
     else:
        return np.nan

In [1327]:
def Euclidean(point1, point2):
    '''
    In this function, it needs two points to calculate the Euclidean distance.
    If the data isn't digited and doesn't equal, we give it scores 50 to distance them because maybe they are nan or difference type. 
    We can use values to control the weight of the non_value data such as the different brands.
    '''
    distance = 0
    
    price = float(point1[0])
    Item_Weight = float(point1[1])
    Product_Dimensions = point1[2]
    Manufacturer_recommended_age = float(point1[3])
    last_time = point1[4]
    title = point1[5]
    describe1 = point1[6]
    describe2 = point1[7]
    describe3 = point1[8]
    describe4 = point1[9]
    describe5 = point1[10]
    
    price2 = float(point2[0])
    Item_Weight2 = float(point2[1])
    Product_Dimensions2 = point2[2]
    Manufacturer_recommended_age2 = float(point2[3])
    last_time2 = point2[4]
    title2 = point2[5]
    describe12 = point2[6]
    describe22 = point2[7]
    describe32 = point2[8]
    describe42 = point2[9]
    describe52 = point2[10]
    if math.isnan(price) or math.isnan(price2):#price's distance is 20
        distance+=30
    else:
        distance+=(float(price)-float(price2))**2
    
    if math.isnan(Item_Weight) or math.isnan(Item_Weight2):#Item_Weight's distance is 20
        distance+=30
    else:
        distance+=(float(Item_Weight)-float(Item_Weight2))**4
    
    if type(Product_Dimensions)==float or type(Product_Dimensions2)==float:
        distance+=30
    else:
        if len(Product_Dimensions) == len(Product_Dimensions2):
            for item_dim in range(len(Product_Dimensions)):
                distance+=(float(Product_Dimensions[0])-float(Product_Dimensions2[0]))**2+(float(Product_Dimensions[1])-float(Product_Dimensions2[1]))**2+(float(Product_Dimensions[2])-float(Product_Dimensions2[2]))**2
        else:
            distance+=30
    
    if math.isnan(Manufacturer_recommended_age) or math.isnan(Manufacturer_recommended_age2):#Item_Weight's distance is 20
        distance+=30
    else:
        distance+=(float(Manufacturer_recommended_age)-float(Manufacturer_recommended_age))**2
    
    if pd.isnull(last_time) or pd.isnull(last_time2) or last_time==np.nan or last_time2==np.nan:#Item_Weight's distance is 20
        distance+=30
    else:
        last_time = str(last_time)
        last_time2 = str(last_time2)
        time1 = last_time.split(' ')[0]
        time2 = last_time2.split(' ')[0]
        distance+= (int(time1)-int(time2))**2
    ##############title
    if type(title)==float or type(title2)==float:#price's distance is 20
        distance+=30
    else:
        title = title.lower()
        title2 = title2.lower()
        pattern = re.compile(r'[\w]+')
        title = re.findall(pattern, title)
        title2 = re.findall(pattern, title2)
        title = remove_stop_word(title)
        title2 = remove_stop_word(title2)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = title.union(title2)
        l1 = []
        l2 = []
        for w in rvector:
            if w in title: l1.append(1)
            else: l1.append(0)
            if w in title2: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        if cosine>0.3:
            distance-=5
        else:
            distance+10
    
    if type(describe1)==float or type(describe12)==float:#price's distance is 20
        distance+=30
    else:
        describe1 = describe1.lower()
        describe12 = describe12.lower()
        pattern = re.compile(r'[\w]+')
        describe1 = re.findall(pattern, describe1)
        describe12 = re.findall(pattern, describe12)
        describe1 = remove_stop_word(describe1)
        describe12 = remove_stop_word(describe12)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = describe1.union(describe12)
        l1 = []
        l2 = []
        for w in rvector:
            if w in describe1: l1.append(1)
            else: l1.append(0)
            if w in describe12: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        if cosine>0.3:
            
            distance-=5
        else:
            distance+10
    
    if type(describe2)==float or type(describe22)==float:#price's distance is 20
        distance+=30
    else:
        describe2 = describe2.lower()
        describe22 = describe22.lower()
        pattern = re.compile(r'[\w]+')
        describe2 = re.findall(pattern, describe2)
        describe22 = re.findall(pattern, describe22)
        describe2 = remove_stop_word(describe2)
        describe22 = remove_stop_word(describe22)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = describe2.union(describe22)
        l1 = []
        l2 = []
        for w in rvector:
            if w in describe2: l1.append(1)
            else: l1.append(0)
            if w in describe22: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        if cosine>0.3:
            
            distance-=5
        else:
            distance+10
            
    if type(describe3)==float or type(describe32)==float:#price's distance is 20
        distance+=30
    else:
        describe3 = describe3.lower()
        describe32 = describe32.lower()
        pattern = re.compile(r'[\w]+')
        describe3 = re.findall(pattern, describe3)
        describe32 = re.findall(pattern, describe32)
        describe3 = remove_stop_word(describe3)
        describe32 = remove_stop_word(describe32)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = describe3.union(describe32)
        l1 = []
        l2 = []
        for w in rvector:
            if w in describe3: l1.append(1)
            else: l1.append(0)
            if w in describe32: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        if cosine>0.3:
            distance-=5
        else:
            distance+10
            
            
    if type(describe4)==float or type(describe42)==float:#price's distance is 20
        distance+=30
    else:
        describe4 = describe4.lower()
        describe42 = describe42.lower()
        pattern = re.compile(r'[\w]+')
        describe4 = re.findall(pattern, describe4)
        describe42 = re.findall(pattern, describe42)
        describe4 = remove_stop_word(describe4)
        describe42 = remove_stop_word(describe42)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = describe4.union(describe42)
        l1 = []
        l2 = []
        for w in rvector:
            if w in describe4: l1.append(1)
            else: l1.append(0)
            if w in describe42: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        
        if cosine>0.3:
            distance-=5
        else:
            distance+10
            
            
    if type(describe5)==float or type(describe52)==float:#price's distance is 20
        distance+=30
    else:
        describe5 = describe5.lower()
        describe52 = describe52.lower()
        pattern = re.compile(r'[\w]+')
        describe5 = re.findall(pattern, describe5)
        describe52 = re.findall(pattern, describe52)
        describe5 = remove_stop_word(describe5)
        describe52 = remove_stop_word(describe52)
        
        
        #Similarity = (A.B) / (||A||.||B||) where A and B are vectors.
        rvector = describe5.union(describe52)
        l1 = []
        l2 = []
        for w in rvector:
            if w in describe5: l1.append(1)
            else: l1.append(0)
            if w in describe52: l2.append(1)
            else: l2.append(0)
        c = 0
        for i in range(len(rvector)):
            c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        if cosine>0.3:
            distance-=5
        else:
            distance+10
    return distance

In [1338]:
x_text_n = 1

In [1339]:
X_test.iloc[x_text_n][0]

nan

In [1340]:
#k=3
k=3
query = X_test.iloc[x_text_n]
distance = knn(X_train, query, k, distance_fn=Euclidean)
#index = [i for dis,i in recomment]

In [1341]:
X_train_result = X_train.copy()
X_train_result['distance'] = distance

In [1342]:
X_test.iloc[x_text_n]

price_list                                                                         NaN
Item_Weight_list                                                                  2.29
Product_Dimensions_list                                               [14.8, 9.5, 2.5]
Manufacturer_recommended_age_list                                                  NaN
last_time                                                                          NaT
title_list                           Inspired Thinkers Magnetic Letters - 240-Piece...
describe1_list                       Spell out fun with more ABCs! Our kids magnets...
describe2_list                       No more puzzled looks over fancy fonts and unn...
describe3_list                       Whether your child is learning in the classroo...
describe4_list                       Teach kids how to organize, sort, and classify...
describe5_list                       Our red and blue magnet letters for kids, alon...
Name: 662, dtype: object

In [1343]:
X_train_result.nsmallest(3, 'distance')

,price_list,Item_Weight_list,Product_Dimensions_list,Manufacturer_recommended_age_list,last_time,title_list,describe1_list,describe2_list,describe3_list,describe4_list,describe5_list,distance
522,NaN,2.29,"[14.8, 9.5, 2.5]",NaN,NaT,Inspired Thinkers Magnetic Letters - 240-Piece...,Spell out fun with more ABCs! Our kids magnets...,No more puzzled looks over fancy fonts and unn...,Whether your child is learning in the classroo...,"Teach kids how to organize, sort, and classify...","Our red and blue magnet letters for kids, alon...",60.000000
753,22.99,2.60,"[14.77, 9.61, 2.52]",4,NaT,JoyNote Classroom Magnetic Letters Kit 234 Pcs...,Ample Letters Set: The sturdy storage box cont...,"Practical Teaching Tools: High-quality, double...",Flawless Details: Each square box has a corres...,High-quality Materials: All letters are made o...,Accompanying Children's Learning: Suitable for...,85.049435
417,25.99,2.60,"[13.6, 8.4, 1.9]",NaN,NaT,Gamenote Classroom Magnetic Alphabet Letters K...,"Colorful, all letters have 8 colors: 234pcs le...",Ample Letters Set: The sturdy storage box cont...,Practical Teaching Tools and Flawless details:...,Good-quality Materials: All letters are made o...,Accompanying children' s learning: Suitable fo...,94.039235


In [1344]:
y_recommand = y_train[y_train.index.isin(X_train_result.nsmallest(3, 'distance').index)]
y_recommand

,number_rating_list,Best_Sellers_Rank_list,url,ASIN_list
417,2476.0,906.0,https://www.amazon.com/Gamenote-Classroom-Magn...,B07NZMJGWB
753,2810.0,1.0,https://www.amazon.com/Gamenote-Classroom-Magn...,B07CL3VJQ2
522,1930.0,1.0,https://www.amazon.com/Magnets-for-kids-magnet...,B07NTWLSXM


In [1345]:
for i in range(len(y_recommand)):
    print(y_recommand.iloc[i,2])

https://www.amazon.com/Gamenote-Classroom-Magnetic-Alphabet-Double-Side/dp/B07NZMJGWB/ref=sr_1_401?dchild=1&qid=1628620372&refinements=p_n_age_range%3A165890011&rnid=165794011&s=toys-and-games&sr=1-401
https://www.amazon.com/Gamenote-Classroom-Magnetic-Letters-Double-Side/dp/B07CL3VJQ2/ref=sr_1_518?dchild=1&qid=1628621231&refinements=p_n_age_range%3A165890011&rnid=165794011&s=toys-and-games&sr=1-518
https://www.amazon.com/Magnets-for-kids-magnetic-letters/dp/B07NTWLSXM/ref=sr_1_443?dchild=1&qid=1628620915&refinements=p_n_age_range%3A165890011&rnid=165794011&s=toys-and-games&sr=1-443


In [1348]:
y_test.iloc[x_text_n, 0]

1930.0

In [1337]:
X_test.iloc[2].index

Index(['price_list', 'Item_Weight_list', 'Product_Dimensions_list',
       'Manufacturer_recommended_age_list', 'last_time', 'title_list',
       'describe1_list', 'describe2_list', 'describe3_list', 'describe4_list',
       'describe5_list'],
      dtype='object')

# 5. accuracy

In [1353]:
def accuracy_amazon(X_train, X_test, y_train, y_test):
    #k=3
    k=3
    total_number_rating_accuracy_number = 0
    total_Best_Sellers_Rank_accuracy_number = 0
    for i in range(len(X_test)):
        query = X_test.iloc[i]
        distance = knn(X_train, query, k, distance_fn=Euclidean)
        X_train_result = X_train.copy()
        X_train_result['distance'] = distance
        y_recommand = y_train[y_train.index.isin(X_train_result.nsmallest(k, 'distance').index)]
        y_test_number_rating = y_test.iloc[i, 0]
        y_test_Best_Sellers_Rank = y_test.iloc[i, 1]
        number_rating_accuracy_number = 0
        Best_Sellers_Rank_accuracy_number = 0
        #If  True number of rating-200 <the predicted number of rateing<True number of rateing+200, we think predict is well
        #If  Best Sellers Rank-10 <the predicted number of Best Sellers Rank<Best Sellers Rank+10, we think predict is well
        for j in range(len(y_recommand)):
            y_recommand_number_rating = y_recommand.iloc[j, 0]
            y_recommand_Best_Sellers_Rank = y_recommand.iloc[j, 1]
            
            if math.isnan(y_test_number_rating) and math.isnan(y_recommand_number_rating):
                number_rating_accuracy_number+=1
                continue
            if math.isnan(y_test_Best_Sellers_Rank) and math.isnan(y_recommand_Best_Sellers_Rank):
                number_rating_accuracy_number+=1
                continue
            if math.isnan(y_test_number_rating) or math.isnan(y_recommand_number_rating):
                continue
            if math.isnan(y_test_Best_Sellers_Rank) or math.isnan(y_recommand_Best_Sellers_Rank):
                continue
            if int(y_recommand_number_rating) < (int(y_test_number_rating)+200) and int(y_recommand_number_rating) > (int(y_test_number_rating)-200):
                number_rating_accuracy_number+=1
            if int(y_recommand_Best_Sellers_Rank) < (int(y_test_Best_Sellers_Rank)+10) and int(y_recommand_Best_Sellers_Rank) > (int(y_test_Best_Sellers_Rank)-10):
                Best_Sellers_Rank_accuracy_number+=1
        total_number_rating_accuracy_number+=number_rating_accuracy_number
        total_Best_Sellers_Rank_accuracy_number+=Best_Sellers_Rank_accuracy_number
        print(f'The number of rating accuracy = {number_rating_accuracy_number/len(y_recommand)}')
        print(f'The Best Sellers Rank accuracy = {Best_Sellers_Rank_accuracy_number/len(y_recommand)}')
        
    print(f'The total number of rating accuracy={total_number_rating_accuracy_number/len(X_test)}')
    print(f'The total Best Sellers Rank accuracy={total_Best_Sellers_Rank_accuracy_number/len(X_test)}')

In [1354]:
accuracy_amazon(X_train, X_test, y_train, y_test)

The number of rating accuracy = 0.3333333333333333
The Best Sellers Rank accuracy = 0.3333333333333333
The number of rating accuracy = 0.3333333333333333
The Best Sellers Rank accuracy = 0.6666666666666666
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.0
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.6666666666666666
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.0
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.3333333333333333
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.0
The number of rating accuracy = 0.3333333333333333
The Best Sellers Rank accuracy = 0.0
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.0
The number of rating accuracy = 0.3333333333333333
The Best Sellers Rank accuracy = 0.3333333333333333
The number of rating accuracy = 0.0
The Best Sellers Rank accuracy = 0.6666666666666666
The number of rating accuracy = 0.6666666666666